In [ ]:
import numpy as np

In [ ]:
def stochastic_rounding(x):

    x_int = x.astype(np.int32)
    x_frac = np.abs(x - x_int)
    rng = np.random.random(x_int.shape)

    # positive
    rounded_pos = np.where(x_frac > rng, x_int + 1, x_int)

    # negative
    rounded_neg = np.where(x_frac > rng, x_int - 1, x_int)

    rounded = np.where(x < 0, rounded_neg, rounded_pos)
    return rounded

x = np.random.normal(0, 5, (5)) 
print("float ", x)
rx = 0
it = 100
for _ in range(it):
    rx += stochastic_rounding(x)

rx = rx.astype(np.float32) / it
print("stchr0", stochastic_rounding(x))
print("stchr1", rx)
print("round ", np.around(x).astype(np.int32))

In [ ]:

def quantize(x, round_stoch = True):
    """ exponentiation and quantization function """

    # just to avoid numerical problems
    eps = 1e-8

    s = np.sign(x)

    abs_x = np.abs(x)

    cliped_abs_x = np.where(abs_x < eps, eps, abs_x)
    cliped_abs_x = np.where(cliped_abs_x > 1, 1, cliped_abs_x)

    # gets the exponent with base 2
    exp = np.log2(cliped_abs_x)

    # round to nearest and cast to int (use stochastic rounding)
    if round_stoch:
        round_exp = stochastic_rounding(exp)
    else:
        round_exp = (np.round(exp)).astype(np.int32)

    # clip to the limit of sx4
    clip_exp = np.where(round_exp < -7, -7, round_exp)

    # stochastic zero ???  


    # convert to float32 again
    qx = s * np.power(2., clip_exp)
    return qx


for _ in range(10):
    x = np.random.normal(0, 0.2, (5)) 
    print("float ", x)
    print("quant0", quantize(x))
    print("quant1", quantize(x))
    print("quant2", quantize(x))    
    print("-------")